In [1]:
%pylab inline
import sys
sys.path.append('../build/Debug')
from pyswarm import pso
from transformations import quaternion_matrix, euler_matrix
from libpso import rotate_point, rotor
np.set_printoptions(precision=6, suppress=True)

Populating the interactive namespace from numpy and matplotlib


In [2]:
rotor(1,0,0,0)

TypeError: No to_python (by-value) converter found for C++ type: vsr::Multivector<vsr::algebra<vsr::metric<4, 1, true>, double>, vsr::Basis<(short)0, (short)3, (short)5, (short)6> >

In [21]:
def generate_dataset(n_points, sigma, mu):
    n_points = n_points
    r = euler_matrix(0.0,0.0,np.pi/2.0)[:3,:3]
#     angles = np.pi * np.random.randn(3)
#     r = euler_to_matrix(angles[0], angles[1], angles[2])[:3,:3]
    points_a = np.random.normal(0.0,0.8,(n_points,3))
    points_a = np.array([point / np.linalg.norm(point) for point in points_a])
    points_b = np.array([np.dot(r,point.reshape(3,1)).reshape(3) + np.array([0.0,0.0,1.0]) for point in points_a])
    # Add noise to the rotated point cloud
    sigma = sigma
    mu = mu
    points_b_noisy = points_b + sigma * np.random.randn(n_points,3) + mu
    points_b_noisy = np.array([point / np.linalg.norm(point) for point in points_b_noisy])
    return (r, points_a, points_b, points_b_noisy)

In [22]:
(r, points_a, points_b, points_b_noisy) = generate_dataset(10, 0.0, 0.0)
r

array([[ 0., -1.,  0.],
       [ 1.,  0.,  0.],
       [-0.,  0.,  1.]])

In [23]:
def myfunc(q):
    q[:4] /= np.linalg.norm(q[:4])
    b = points_b
    b_ = np.array([rotate_point(q.reshape(8,1), point.reshape(3,1)).reshape(3) for point in points_a])
    return np.sum([np.linalg.norm(pa - pb) for pa, pb in zip(b, b_)])

def myfunc2(q):
    q = np.array(q) / np.linalg.norm(q)
    r = quaternion_matrix(q.reshape(4))[:3,:3]
    b = points_b
    b_ = np.array([np.dot(r,point.reshape(3,1)).reshape(3) for point in points_a])
    return np.sum([np.linalg.norm(pa - pb) for pa, pb in zip(b, b_)])

In [34]:
lb = [-1] * 8
# lb = [0,0,0,0]
ub = [1] * 8
q, d = pso(myfunc, lb, ub, 
           swarmsize=250, 
           maxiter=1000, 
           minstep=1e-12, 
           minfunc=1e-12,
           omega=0.75,
           phip=0.75,
           phig=0.75,
           debug=False)
print(d)
print(q.reshape(8,1))

Stopping search: Swarm best position change less than 1e-12
2.17344408273e-11
[[ 0.707107]
 [-0.707107]
 [-0.      ]
 [ 0.      ]
 [ 0.      ]
 [ 0.      ]
 [-1.      ]
 [-0.292893]]


In [10]:
m = q
a = -0.5 * m[0] * np.eye(3) + \
np.array([0.0, -m[1], m[2], 
          m[1], 0.0, -m[3],
         -m[2], m[3], 0.0]).reshape(3,3)

b = np.array([m[4], m[5], m[6]]).reshape(3,1)
print a
np.linalg.solve(a,b)

[[ 0.35349  -0.707234  0.000366]
 [ 0.707234  0.35349  -0.000422]
 [-0.000366  0.000422  0.35349 ]]


array([[ 1.200293],
       [-0.400043],
       [ 0.17935 ]])

In [25]:
pso?

In [8]:
rotor = np.array([cos(pi/4),-sin(pi/4),1.0,1.0, 1.0,1.0,1.0]).reshape(7,1)
# rotor = np.array([1.0, 0.0, 0.0,0.0]).reshape(4,1)
vec = np.array([1.0, 0.0, 0.0]).reshape(3,1)
rotate_point(rotor, vec)

RuntimeError: rotor: Array must have shape (8,1)

In [ ]:
rotor[:4] /= 3.0
rotor